## Check GPU device

In [2]:
!nvidia-smi

Mon Jul 10 07:50:05 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:3A:00.0 Off |                    0 |
| N/A   26C    P0    39W / 300W |      0MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Autoreload packages on change

In [3]:
import IPython
%load_ext autoreload

%autoreload 2

## Libs and pacakges

In [4]:
import pandas as pd

import os
import os.path as osp
import sys

import torch


In [5]:
# import implemented modules from our source folder
sys.path.append("../src/")

In [6]:
os.chdir('../src')

In [7]:
os.getcwd()

'/ocean/projects/cis230036p/cmanouan/JEPSAM/AEJEPS/src'

In [8]:
import models
from utils.parser import load_config, parse_args
import utils.model_utils as model_utils
from utils.ae_resnet import get_configs, ResNetEncoder, ResNetDecoder

from dataloader import get_dataloaders, SimpleTokenizer, JEPSAMDataset
import vocabulary as vocab

/jet/home/cmanouan/miniconda3/envs/jeps/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
cfg = load_config(config_file_path="../configs/aejeps_cfg.yaml")

In [15]:
cfg.DATASET

{'PATH': '../../../dataset/',
 'VOCABULARY_SIZE': 65,
 'IMAGE_SIZE': 224,
 'SOS': 48,
 'EOS': 46,
 'TOKENIZER_PATH': '../../../dataset/jeps_tokenizer.json',
 'ACTION_DESC_MAX_LEN': 64,
 'CMD_MAX_LEN': 128,
 'VALIDATION_PCT': 0.3,
 'NUM_COMMANDS': 46,
 'TRAIN_TFMS': {'Resize': {'height': 224, 'width': 224},
  'RandomBrightnessContrast': {'p': 0.2}},
 'TEST_TFMS': {'Resize': {'height': 224, 'width': 224}}}

## Dataset and Data loader

In [16]:
tdf = pd.read_csv(
    osp.join(cfg.DATASET.PATH, "updated_train.csv")
)

tdf.head()

,sample_ID,in_state,goal_state,validator,action_description,motor_cmd,len_action_desc,len_motor_cmd
0,1005,0,9,amihretu,put the :BOTTLE to the left of :BOTTLE,:BOTTLE BLUE POSE-9 :BOTTLE RED POSE-2 :BOTTLE...,8,11
1,1011,0,9,amihretu,move the :BOTTLE left,:BOTTLE BLUE POSE-3 :BOTTLE #'*leftward-trans...,4,8
2,1012,0,9,amihretu,put the :BOTTLE to the right of :MUG,:BOTTLE BLUE POSE-7 :MUG RED POSE-3 :BOTTLE #...,8,11
3,1013,0,9,amihretu,shift the :CUP backwards,:CUP RED POSE-4 :CUP #'*backward-transformati...,4,8
4,1015,0,9,amihretu,shift the :BOTTLE forwards,:BOTTLE GREEN POSE-3 :BOTTLE #'*forward-trans...,4,8


In [17]:
train_dl, _ = get_dataloaders(
    train_df=tdf,
    cfg=cfg,
    # dataset_module=JEPSAMDataset
)


INFO:root:Prepared 1343 training samples and 627 validation samples 


In [28]:
for data in train_dl:
    in_state, goal_state, ad, cmd, ad_lens, cmd_lens = data
    pass
    break
    
    
in_state.shape, goal_state.shape, ad.shape, cmd.shape, ad_lens.shape, cmd_lens.shape

(torch.Size([16, 3, 224, 224]),
 torch.Size([16, 3, 224, 224]),
 torch.Size([16, 1, 10]),
 torch.Size([16, 1, 11]),
 torch.Size([16]),
 torch.Size([16]))

## Modeling

In [30]:
model = models.JEPSAM(cfg=cfg).cuda()

model

JEPSAM(
  (encoder): JEPSAMEncoder(
    (embedding): Embedding(65, 256)
    (image_feature_extractor): Sequential(
      (0): ResNetEncoder(
        (conv1): Sequential(
          (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (conv2): EncoderBottleneckBlock(
          (00 MaxPooling): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
          (01 EncoderLayer): EncoderBottleneckLayer(
            (weight_layer1): Sequential(
              (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU(inplace=True)
            )
            (weight_layer2): Sequential(
              (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias

In [31]:
per_image_rec, goal_image, lang_out, motor_out = model(data)

In [32]:
per_image_rec.shape, goal_image.shape, lang_out.shape, motor_out.shape

(torch.Size([16, 3, 224, 224]),
 torch.Size([16, 3, 224, 224]),
 torch.Size([16, 11, 65]),
 torch.Size([16, 11, 65]))

## Training procedure

In [44]:
! python run.py

INFO:root:Prepared 1389 training samples and 581 validation samples 
/jet/home/cmanouan/miniconda3/envs/jeps/lib/python3.8/site-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Started Autoencoder JEPS training
Training [Epoch 1/100]:   0%|                            | 0/86 [00:07<?, ?it/s]
Traceback (most recent call last):
  File "run.py", line 17, in <module>
    main()
  File "run.py", line 11, in main
    run_AEJEPS(args=args, cfg=cfg)
  File "/ocean/projects/cis230036p/cmanouan/JEPSAM/AEJEPS/src/train.py", line 224, in run_AEJEPS
    train_loss  = train(
  File "/ocean/projects/cis230036p/cma